# 3. Manejando Netcdf

Para manejar netcdfs usando python haremos uso extensivo de los paquetes estrella de la plataforma Pangeo. Pangeo es un esfuerzo de la comunidad que promueve la reproductibilidad, escalamiento y transparencia en la ciencia.

El paquete de cabecera para el manejo de netcdfs será [xarray](http://xarray.pydata.org/en/latest/). Xarray extiende las capacidades de numpy en el manejo de datos n-dimensionales introduciendo etiquetas en forma de dimensiones, coordenadas y atributos que facilitan las operaciones sobre los mismos. El motor para las operaciones entre fechas es proporcionado por pandas y la escalabilidad de los datos es proporcionado por dask, lo cual convierte a este paquete en una herramienta poderosa.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

## 3.1 Estructuras de xarray

xarray cuenta con dos estructuras simples que son fundamentales comprender para un mejor uso del paquete. Esta es una brevisima descripción sobre algunos conceptos claves que se deben tener presentes, mayor informacion la pueden encontrar siempre en la [documentacion](http://xarray.pydata.org/en/stable/data-structures.html#)

### 3.1.1 DataArray

Un `DataArray` es virtualmente igual a un arreglo n-dimensional de numpy que cuenta con etiquetas en sus dimensiones, siendo el elemento base de xarray. Para denifinir un `DataArray` necesitaremos proporcionar, como mínimo, una arreglo de elementos numéricos.

In [ ]:
xr.DataArray(np.random.randn(10,40))

Como podemos observar, xarray automaticamente asignó nombres a las dimensiones y nos informa que no encuentra coordenadas asociadas a estas dimensiones. Ahora procederemos a declarar un `DataArray` con la información que falta

In [ ]:
# Creamos un poco de data falsa
lat = np.arange(-90, 90, 0.25)
lon = np.arange(0, 360, 0.25)
llon, llat = np.meshgrid(lon, lat)
data = np.sin(llat**2-llon**2)

# Creamos el DataArray
xarr = xr.DataArray(data, coords=[lat,lon], dims=["lat","lon"])
xarr

Nuestro `DataArray` ha sido creado dentro de la variable `xarr` satisfactoriamente con las dimensiones y coordenadas asignadas correctamente. Ahora podemos acceder a los métodos que xarray ofrece para sus objetos.

In [ ]:
xarr.plot()

### 3.1.2 Dataset

Un `Dataset` es la representación en la memoria del sistema de un archivo netcdf. Al igual que en pandas un `DataFrame` es un conjunto de `Series`, un `Dataset` es un conjunto de `DataArray`. Su declaración es un poco más extensa que un `DataArray` pero a cambio proporciona un contenedor a varias variables.

## 3.2 Datos Reales

Para cargar datos en formato netcdf a una variable, se hacer uso de la función `open_dataset` (en el caso de tener 1 solo archivo) o `open_mfdataset` en caso de tener muchos archivos

In [ ]:
ersstv5 = xr.open_dataset('https://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc')
ersstv5

Como podemos observar, la información mostrada es muy similar a lo que uno obtendría al usar `ncdump -h` en la linea de comandos sobre un archivo netcdf. Esta variable, al ser un objeto de xarray, nos da acceso a una variedad de métodos muy útiles para el procesamiento y manejo de dato en geociencias.

### Calculo del ONI
Como ejemplo, vamos a calcular el ONI (Oceanic Nino Index) y compararemos nuestros resultados con los resultados oficiales de la NOAA. El ONI esta definido como la media corrida de 3 meses sobre el indice del Niño 3.4 (5°S-5°N / 170°W-120°W)

In [ ]:
nino34 = ersstv5.sst.sel(lat=slice(5,-5), lon=slice(190, 240))
nino34

Podemos acceder a algunas funciones utiles de numpy que fueron adaptadas para trabajar con las etiquetas de xarray

In [ ]:
nino34 = nino34.mean(dim=['lat', 'lon'])

- #### Calculamos la climatología
Para realizar este cálculo, vamos a tomar como periodo base 1986-2015 como fines ilustrativos. En el cálculo actual, el periodo de la climatología cambia cada 5 años segun los criterios considerados por el centro de predicción del clima de la NOAA ([ref](https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_change.shtml))

In [ ]:
nino34_clim = nino34.sel(time=slice("1986-01-01", "2015-12-31")).groupby("time.month").mean(dim='time')
nino34_clim

In [ ]:
fig,ax = plt.subplots()
nino34_clim.plot(ax=ax)
ax.set_yticks(np.arange(26,28.1,0.2))
ax.set_ylim(26,28)
ax.grid(ls='--')

- #### Calculamos la anomalía
Utilizaremos una sintaxis similar para restar las climatologías a los valores absolutos

In [ ]:
nino34_anom = nino34.groupby('time.month') - nino34_clim
nino34_anom

Calculamos el promedio de las anomalías en la región y aplicamos una media movil de 3 meses

In [ ]:
ONI = nino34_anom.rolling(time=3, center=True).mean(dim='time')
ONI

Ahora debemos truncar los resultados hasta el segundo decimal. Para lograr esto xarray ofrece una poderosa forma de aplicar funciones de numpy sobre objetos de xarray. La documentación siempre es de gran ayuda al manejar este tipo de funciones complejas ([ref](http://xarray.pydata.org/en/latest/generated/xarray.apply_ufunc.html))

In [ ]:
def truncate_decimals(xrobj, decimals=2):
    return xr.apply_ufunc(np.around, xrobj, kwargs={'decimals':decimals})

In [ ]:
ONI = truncate_decimals(ONI)

xarray es capaz de crear etiquetas para nuestros ejes si es que encuentra los atributos necesarios. Para poder usar esta caracteristica, debemos de usar la [convención CF](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/build/ch03s03.html) al momento de declarar nuestros atributos

In [ ]:
ONI.attrs['long_name'] = "SST Anomaly"
ONI.attrs['units'] = "degrees C"

In [ ]:
ONI.plot()

xarray usa matplotlib como motor para realizar los gráficos. Si bien provee un metodo rápido y sencillo para este fin (`.plot`), este no está limitado y tiene acceso a las funciones de personalización que matplotlib provee. Para esto necesitamos declarar una figura y un eje usando los métodos de matplotlib primero, esto servirá para indicarle a xarray donde deberá colocar nuestro gráfico usando el argumento `ax`.

In [ ]:
# Colocamos el tiempo que queremos plotear 
plot_data = ONI.sel(time=slice("2008-01-01","2019-12-31"))

fig, ax = plt.subplots(dpi=200)

plot_data.plot(ax=ax, c='k', lw=0.5)
ax.set_ylim(-4, 4)
ax.grid(ls='--',lw=0.5)

Usando algunas funciones de matplotlib podemos crear un gráfico listo para publicación.

In [ ]:
# Colocamos el tiempo que queremos plotear 
plot_data = ONI.sel(time=slice("2008-01-01","2019-12-31"))

# Declaramos la figura y los ejes
fig, ax = plt.subplots(dpi=200)

ax.fill_between(plot_data.time.data,plot_data.data, 0.5, where = plot_data > 0.5, color='red', interpolate=True, lw=0, label="ONI > 0.5")
ax.fill_between(plot_data.time.data,plot_data.data, -0.5, where = plot_data < -0.5, color='blue', interpolate=True, lw=0, label="ONI < 0.5")
plot_data.plot(ax=ax, c='k', lw=0.5)

minx = plot_data.time.min().data
maxx = plot_data.time.max().data
ax.hlines(0.5, minx, maxx, lw=0.5, linestyles='--')
ax.hlines(-0.5, minx, maxx, lw=0.5, linestyles='--')

ax.set_ylim(-4, 4)
ax.set_xlim(minx, maxx)

ax.text(0.99, 0.01, "Clim 1981-2010", fontsize=5, horizontalalignment='right', transform=ax.transAxes)

ax.set_title("ERSSTv5 ONI")
ax.grid(ls='--',lw=1, alpha=0.3)
ax.legend()

## A tomar en cuenta
Si bien hicimos el cálculo de la manera correcta, los valores difieren de los mostrados por la NOAA ([ref](https://www.cpc.ncep.noaa.gov/data/indices/3mth.nino34.81-10.ascii.txt)).

In [ ]:
ONI_noaa = pd.read_fwf('https://origin.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt').rename(columns={'ANOM':'ONI'})
ONI_noaa.head()

Arreglamos un poco los datos para agregar fechas como indices del dataframe

_Nota:_ Por fines practicos estamos colocando los rangos de tiempos fijos ya que conocemos la extensión de nuestra data. Lo recomendable es buscar una forma de poder inferir automaticamente estos rangos conforme los datos se vayan actualizando

In [ ]:
time_series = pd.date_range('1950-01-01','2019-07-01',freq=pd.offsets.MonthBegin())
time_series

In [ ]:
ONI_noaa_ts = ONI_noaa.set_index(time_series)[['ONI']]
ONI_noaa_ts.head()

Podemos hacer un gráfico rápido del dataframe para revisar que estamos haciendo las cosas bien

In [ ]:
ONI_noaa_ts.query("index>='2008-01-01'").plot()

Ahora convertimos nuestra serie de tiempo de pandas a xarray para facilitar la manipulación de los datos.

Es en este momento que debemos de recordar las similitudes entre xarray y pandas. Ambos paquetes cuenta con estructuras básicas: `DataArray` y `Dataset`(xarray), `Series` y `DataFrame` (pandas).

xarray es capaz de crear un `DataArray` usando un arreglo de numpy, pero no se limita a ese tipo de objetos, sino tambien puede aceptar una `Series` de pandas y construir un objeto de xarray; de la misma forma, al ser un `Dataset` una agrupación de `DataArray`s (ya pueden ver por donde va esto), puede aceptar un `DataFrame` el cual es una agrupación de `Series`.

In [ ]:
xONI_noaa = xr.DataArray(ONI_noaa_ts.query("index>='2008-01-01'")['ONI'],dims=('time'))
xONI_noaa

In [ ]:
diff = xONI_noaa - ONI.sel(time=xONI_noaa.time)
fig, ax = plt.subplots()
diff.sel(time=slice('2015-01-01',None)).plot(ax=ax)
ax.set_ylim(-0.5,0.5)

La razón de este error es debido a que la base de datos usada parece tener una ligera variación en los datos absolutos, lo cual no afecta tanto a la climatología pero si a la anomalía calculada. Estos datos venian en un netcdf completo el cual fue facil de manipular usando xarray, ahora usaremos los mismo datos pero de IRI library.

In [ ]:
ersstv5_iri = xr.open_dataset('http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NCDC/.ERSST/.version5/.sst/dods',decode_times=False)
ersstv5_iri

Como podemos observar, las dimensiones no tienen los nombres convencionales (lat, lon, time) y el tiempo no pudo ser entendido por xarray ya que las unidades son en meses desde 1960 (lo convencional es tener estas unidades en días).

Para arreglar este problema, debemos de cambiar el calendario en el que se encuentra nuestra variable tiempo a `360_day` para que pueda ser entendido por el paquete [cftime](https://unidata.github.io/cftime/api.html) (_[GH issue](https://github.com/Unidata/cftime/pull/69)_)

In [ ]:
ersstv5_iri.T

In [ ]:
ersstv5_iri.T.attrs['calendar'] = '360_day'
# Ahora le decimos a xarray que interprete los tiempos
ersstv5_iri = xr.decode_cf(ds)
ersstv5_iri

Sin embargo, nuestra dimensión tiempo es un objeto de cftime, por lo que la manipulación de fechas estará limitada en cierta forma. Esta forma de tratar con fechas no convencionales se puede encontrar en la [documentación](http://xarray.pydata.org/en/stable/weather-climate.html#non-standard-calendars-and-dates-outside-the-timestamp-valid-range)

In [ ]:
ersstv5_iri['T'] = ersstv5_iri.indexes['T'].shift(-15,'D').to_datetimeindex()
ersstv5_iri['T']

Como queremos manejar nuestros datos de una manera más entendible, vamos a asignar mejores nombres a las dimensiones

In [ ]:
ersstv5_iri = ersstv5_iri.rename({'X':'lon','Y':'lat','T':'time'})
ersstv5_iri

Ahora procedemos a realizar los calculos para la región el Niño 3.4 como se hizo [previamente](#Calculo-del-ONI)

In [ ]:
nino34_iri = ersstv5_iri.sst.sel(lat=slice(-5, 5), lon=slice(190, 240)).mean(dim=['lat','lon'])
nino34_iri_clim = nino34_iri.sel(time=slice("1986-01-01", "2015-12-30")).groupby("time.month").mean(dim='time')
nino34_iri_anom = nino34_iri.groupby('time.month') - nino34_iri_clim
ONI_iri = nino34_iri_anom.rolling(time=3, center=True).mean(dim='time')
ONI_iri = truncate_decimals(ONI_iri)

In [ ]:
diff = xONI_noaa - ONI_iri.sel(time=xONI_noaa.time)
fig, ax = plt.subplots()
diff.sel(time=slice('2015-01-01',None)).plot(ax=ax)
ax.set_ylim(-0.5,0.5)